In [80]:
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
import sys
sys.path.append("..")
from Vocabulary import *
import math
import numpy as np

In [81]:
has_tag_Rep = True
embedding_path = r'E:\\embeddings\\test200dWTags'
embedding_name = r'pos_w_tags_100w_only200Dims'
dimensions = 200

In [82]:
import numpy as np
import tensorflow as tf
id_dict = {}
word_dict = {}

matrix = []
full_path = os.path.join(embedding_path,embedding_name)
with open(full_path, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

302519


In [83]:
#find all variants of a word, without the tag rep
def group_all_words(id_dict):
    groups = {}
    for word in id_dict:
        #print(word)
        if is_tagged_word(word):
            #print('is tagged')
            baseword = word.split(chr(4))[0]
            #print(baseword)
            if baseword in groups:
                groups[baseword].append(word)
            else:
                groups[baseword] = [word]
        elif word.startswith(chr(4)):#exclude tags
            pass
        else:
            if word in groups:
                groups[word].append(word)
            else:
                groups[word] = [word]
    return groups

def is_tagged_word(word):
    return not word.startswith(chr(4)) and str(chr(4)) in word

groups = group_all_words(id_dict)
print(len(groups))
print(groups['duck'])
print(groups['the'])
print(groups['bank'])
print('---------')
print(str(groups)[:100])

245056
['duck\x04noun', 'duck\x04propn', 'duck\x04verb']
['the\x04noun', 'the\x04det', 'the\x04punct', 'the\x04adv', 'the\x04adj', 'the\x04propn']
['bank\x04noun', 'bank\x04adj', 'bank\x04propn', 'bank\x04verb']
---------
{'rubella': ['rubella\x04noun', 'rubella\x04propn'], 'kramer': ['kramer\x04propn'], 'diabolical': ['


In [84]:
print(groups['rubella'])

['rubella\x04noun', 'rubella\x04propn']


In [85]:
unfiltered_vocab = Vocabulary()
unfiltered_vocab.load('E:\\embeddings\\test200dWTags\\unfilteredpos_vocab')

In [88]:
#get all the factors

In [89]:
def get_count(word):
    return unfiltered_vocab.word_frequency[word]

In [ ]:
class Polygon(ABC):
 
    @abstractmethod
    def noofsides(self):
        pass

In [90]:
#requires tag rep
def mult_strat_weighted(word_groups,normalise_before_mult):
    emb = {}
    tmp_matrix = matrix
    if(normalise_before_mult):
        tmp_matrix = tf.nn.l2_normalize(matrix,axis = 1)
    for word_key in groups:
        #print(word_key)
        emb_sum = 0
        
        baseword = -1#should throw an error if not overwritten
        base_count = 0
        for word in word_groups[word_key]:
            c = get_count(word)
            base_count += c
        
        for word in word_groups[word_key]:
            #print(word)
            if chr(4) in word:
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                id_tag  = id_dict[chr(4)+tag]
                #print(word, tag)
                emb_sum += (get_count(word)/base_count)*(tmp_matrix[id_word] * tmp_matrix[id_tag])
            else:#only in mixed
                emb_sum += tmp_matrix[id_dict[word]] * (get_count(word)/base_count)
        emb[baseword] = emb_sum
        #print(baseword,emb_sum)
    return emb
        
emb = mult_strat_weighted(groups,False)
#print(str(emb)[:100])
#print(emb('the'))

In [91]:
print(emb['the'])

[-0.01633265  0.00186513  0.00497749 -0.00131566 -0.07106869  0.02760165
  0.01533051  0.00023506 -0.00372713  0.00514028 -0.05852658  0.01161702
  0.0101486   0.00505297  0.06273965  0.00585055  0.02269495 -0.06084702
  0.009542    0.02628339 -0.00048417 -0.04856513 -0.00156393  0.0614124
 -0.00214901 -0.02973287 -0.01237914  0.00297455 -0.0080978   0.00236366
 -0.01680189 -0.02538641  0.05166951  0.02206524 -0.00996914  0.00914754
 -0.00440594 -0.03832731 -0.07844426 -0.06790831  0.04124945  0.00180786
 -0.0269876   0.02316665 -0.01693314  0.02048577 -0.00392056  0.00536526
 -0.03802988  0.02207552 -0.03386884 -0.00291344  0.01073835  0.01552975
  0.0361063   0.03101007 -0.00180485  0.01077286 -0.01129019  0.0421184
  0.02420453 -0.0166186  -0.01576072 -0.0150183  -0.00551208 -0.00185257
  0.00304361 -0.03313197  0.04293452 -0.02939568 -0.04096829  0.00576442
 -0.05428303 -0.04265325 -0.01899032  0.00986378 -0.05755935 -0.05684856
  0.00309197  0.02142445 -0.08111324  0.00109211 -0.0

In [92]:
#requires tag rep
def strat_noTagRep(word_groups,normalise_before_mult):
    tmp_matrix     = matrix
    if(normalise_before_mult):
        tmp_matrix = tf.nn.l2_normalize(matrix,axis = 1)
    
    emb = {}
    for word_key in groups:
        
        baseword   = -1 #should throw an error if not overwritten
        base_count =  0
        for word in word_groups[word_key]:
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            #print(word)
            if chr(4) in word:
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                id_tag  = id_dict[chr(4)+tag]
                #print(word, tag)
                emb_sum += (get_count(word)/base_count)* tmp_matrix[id_word]
            else:#only in mixed
                emb_sum +=(get_count(word)/base_count) * tmp_matrix[id_dict[word]]
        emb[baseword] = emb_sum
        #print(baseword,emb_sum)
    return emb
        
emb = mult_strat_noTagRep(groups,False)

In [93]:
print(emb['the'])

[ 0.83655888  0.56230641  1.31466868  0.75338636  0.88259477  0.54048636
  0.66813626  0.24701116  1.08327434  0.73842778  0.93343065  0.58456334
  0.7408845   0.62628036  1.04259515  0.80160135  1.280772    0.67505019
  1.02274889  0.8647537   0.2768535   0.78671927 -0.07325811  0.89054524
  0.79636452  0.9597792   0.72155377  0.25776857  0.59434872  0.45070298
  0.82455262  0.7198299   1.2681247   1.09165162  0.56565395  0.54518698
  0.85127129  0.82685436  1.01986472  0.82717568  0.81580072  0.25343165
  0.71523798  0.80283276  0.66233341  0.41603753  1.03093484  0.8292028
  0.92813965  1.39479177  0.80875336  0.15275364  0.7153622   0.42662276
  0.46856623  0.48922916  0.12875231  1.09962834 -0.2594594   0.74718473
  0.55902573  0.87082431  0.79948158  0.77297602  0.08179791  0.31714618
  1.03793356  0.79005332  0.84827023  0.79805702  0.82308105  0.77717281
  0.73482255  0.90558642  0.36310445  0.22822998  1.20710833  0.77715428
  0.22117303  0.68976168  1.01707578  0.20818995  0.

In [94]:
#requires tag rep
def add_strat_TagRep(word_groups,normalise_before_mult):
    tmp_matrix     = matrix
    if(normalise_before_mult):
        tmp_matrix = tf.nn.l2_normalize(matrix,axis = 1)
    
    emb = {}
    for word_key in groups:
        
        baseword   = -1 #should throw an error if not overwritten
        base_count =  0
        for word in word_groups[word_key]:
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            #print(word)
            if chr(4) in word:
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                id_tag  = id_dict[chr(4)+tag]
                #print(word, tag)
                emb_sum += (get_count(word)/base_count)*(tmp_matrix[id_word] + tmp_matrix[id_tag])
            else:#only in mixed
                emb_sum += (get_count(word)/base_count) * tmp_matrix[id_dict[word]]
        emb[baseword] = emb_sum
        #print(baseword,emb_sum)
    return emb

In [95]:
print(emb['the'])

[ 2.76229148  2.51590141 -0.0420942   3.04173901  2.64908255  1.87738493
  1.45163523  1.19443803  0.6250927   1.69642911  1.61737342  1.52952278
  2.29894397  2.15844273 -0.14075882  1.39544763  2.77200947  3.19888065
  3.13699791  1.70949071  2.79992501  1.41123817  0.95375259  3.11368731
  2.20464367  1.77384376  2.96637402  0.31623668  0.7891416   2.83012274
  2.97160488  1.35648132  2.75739571  2.45551568  1.0200732   2.98060491
  2.04349855  1.16320939  3.09564091  2.30113335  3.00935562  3.09190175
  2.54307032  0.79285289  1.93104478  3.39580113  2.69059502  0.85931519
  2.59802213  1.30132434  2.67013605  0.74340426  2.07360825  2.54885857
  2.48511375  3.35046838  1.12360115  3.55643519 -0.87518359  2.99375467
  1.97027142  2.84140825  1.9278737   2.09939259  1.63945015  2.47593054
  1.59633541  2.1428813   3.37755092  2.96957291  2.53867023  2.52024722
  2.08452366  2.93738024  1.58296975  1.72754244  4.53467243  1.60923323
  2.46810958  1.7468284   2.7909703   1.55511231  1